### Глава 1: Простая симуляция наброса объектов в очередь с обработчиком в виде "черного ящика"

Если набрасывать в очередь среднее количество обработанных элементов:


In [ ]:
def draw_data(data):
    fig = plt.figure()
    fig.set_size_inches(15,10)
    axes = fig.add_axes([0,0,1,1])
    
    # длина очереди после обработки
    axes.plot(data[:,0], data[:,4], label='Queue Length', color='red')

    # интерполяция на полиномах функции длины очереди от времени
    z = np.polyfit(data[:,0], data[:,4], 15)
    p = np.poly1d(z)
    axes.plot(data[:,0], p(data[:,0]), label='Trend', color='blue')

    # к какой длине очереди стремимся
    axes.plot(data[:,0], data[:, 1], label='Taget', color='green')

    axes.legend()

    plt.show()
    

In [ ]:
import numpy as np

def open_loop(p, tm = 5000):
    def target(t):
        return 4.5 # 4.7
    
    vals = []
    for t in range(tm):
        u = target(t)
        y = p.work(u)
        
        vals.append([t, u, 0, u, y])
    
    return np.asarray(vals)

In [ ]:
from buffer import Buffer
import matplotlib.pyplot as plt

p = Buffer(50, 10)

data = open_loop(p, 500)

draw_data(data)

Если использовать моментальную и кумулятивную ошибки:


In [ ]:
from buffer import Buffer
from controller import Controller

def closed_loop(c, p, tm = 5000):
    
    def setpoint(t):
        if (t < 100): return 0
        if (t < 200): return 25
        if (t < 300): return 50
        return 10
    
    vals = []
    
    y = 0
    for t in range(tm):
        r = setpoint(t)
        e = r - y
        u = c.work(e)
        y = p.work(u)
        
        vals.append([t, r, e, u, y])
    
    return np.asarray(vals)

c = Controller(1.25, 0.01)
p = Buffer(50, 10)

data = closed_loop(c, p, 500)

draw_data(data)